In [ ]:
import pickle
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import copy

from models_language_model_style import *
from utils_language_model_style import *

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "2"


dataset_path = 'dataset/SASRecM1M/item_only_full_5U_LM.pkl'
train_set, valid_set, test_set, vocab_u, vocab_i = load_dataset_amazon(dataset_path)

test_dataset = DatasetWithMask(test_set, vocab_u, vocab_i, max_len=201)

# for data in test_set:
#     if data[0]=='A26KXNN6H1IL0I':
#         print(data)
#         break


# test_dataset = DatasetWithMaskProbNegSample([data], vocab_u, vocab_i, max_len=100, num_neg=100)

dataloader_test = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=10)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
import time


model = torch.load('xxxxx/logs_5U_EvalAll=1/xxxxx/model_training/best_model.pt')

model.eval()

total_loss = 0.
total_aux_loss = 0.
start_time = time.time()
for batch, item in enumerate(dataloader_test):

    user_id_torch, seq_torch_pad, target_id_torch, target_id_neg_torch, mask_base, key_mask, pos_id_torch_pad, loss_mask = item

    seq_torch_pad = seq_torch_pad.to(device)
    mask_base = mask_base[0].to(device)
    key_mask = key_mask.to(device)

    target_id_torch = target_id_torch.to(device)
    target_id_neg_torch = target_id_neg_torch.to(device)

    pos_id_torch_pad = pos_id_torch_pad.to(device)

    user_emb = model(seq_torch_pad, mask_base, key_mask, pos_id_torch_pad)  # [batch_size, length, uinterest, dim]

    item_emb_p = model.se(target_id_torch).unsqueeze(2)  # [batch_size, length, 1, dim]
    item_emb_n = model.se(target_id_neg_torch).unsqueeze(2)  # [batch_size, length, 1, num_neg, dim]

    # aux_loss
    activ_matrix = model.activ_matrix[:, :, :, 0]  # [batch_size, length, ninterest]
    #         print(activ_matrix[0, :, :])
    
    
    print(user_id_torch)
    len_batch = 200 - key_mask.sum(dim=-1)
    print(len_batch)
    break

In [ ]:
print(activ_matrix.shape)
%config InlineBackend.figure_format = 'retina'

from IPython import display
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

heat_map = activ_matrix[:, :, :].permute(0, 2, 1)

heatmap = heat_map.cpu().detach().numpy()


In [ ]:
x = 2
heatmap_x = heatmap[x, :, :]
length = len_batch[x]
# plt.axis('off')

ig, ax=plt.subplots()
im =ax.imshow(heatmap_x[:, :length], interpolation='none')

plt.yticks([])
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)
plt.savefig('assignment_matrix_M1M_2I_%d'%x+'.png', dpi=600, bbox_inches='tight')
# plt.show()

# plt.yticks([])
# plt.xticks(range(length)[::10], range(1, length+1)[::10])
# plt.imshow(heatmap_x[:, :length])
# heat_map[:, :-10].shape